### Импорты

In [1]:
import os
import sys
sys.path.insert(0, '../')
import pandas as pd

from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
from colbert import Indexer, Searcher

### Подготовка моделей

In [2]:
pth_models = "/home/sondors/Documents/price/ColBERT_data/18_categories/test/models_18_categories.csv"
df_models = pd.read_csv(pth_models, sep=";")
df_models

/tmp/ipykernel_26882/17807022.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_models = pd.read_csv(pth_models, sep=";")


,model_id,average_price,name,brand_name,full_name,comment,category_name,category_id
0,623742,NaN,920-005619,Logitech,Logitech 920-005619,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
1,721952,NaN,Zipper Bag,Hama,Hama Zipper Bag,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
2,721970,NaN,CC-3064,Nokia,Nokia CC-3064,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
3,751488,990.0,CKS-X7/R,Sony,Sony CKS-X7/R,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
4,751989,NaN,EP-031023,Era Pro,Era Pro EP-031023,NaN,"чехлы, обложки для гаджетов (телефонов, планше...",3994
...,...,...,...,...,...,...,...,...
103209,7049424,16459.0,MD-108,Mivo,Mivo MD-108,NaN,портативная акустика,3904
103210,7049425,8812.0,MD-165,Mivo,Mivo MD-165,NaN,портативная акустика,3904
103211,7049426,4240.0,Boost 20W,Rocket,Rocket Boost 20W,NaN,портативная акустика,3904
103212,7049427,2990.0,Motion 10W,Rocket,Rocket Motion 10W,NaN,портативная акустика,3904


In [3]:
def prepare_tsv(df, pth, category_id):
    """
    Делим модели по full_name и соответствующие им model_id на два tsv файла 

    {category_id}_models.tsv:

    0   model0
    1   model1
    2   model2

    {category_id}_models_id.tsv:

    0   model_id0
    1   model_id1
    2   model_id2
    
    """

    def df_split(df):
        df1 = pd.DataFrame()
        df1["id"], df1["full_name"] = [i for i in range(len(df))], df["full_name"]
        
        df2 = pd.DataFrame()
        df2["id"], df2["model_id"] = [i for i in range(len(df))], df["model_id"]

        return df1, df2
    
    models, models_id = df_split(df)
    models.to_csv(os.path.join(pth,f"{category_id}_models.tsv"), sep='\t', header=False, index=False)
    models_id.to_csv(os.path.join(pth,f"{category_id}_models_id.tsv"), sep='\t', header=False, index=False)

categories = [
    "диктофоны, портативные рекордеры",
    "электронные книги",
    "автомобильные телевизоры, мониторы",
    "смарт-часы и браслеты",
    "портативные медиаплееры",
    # "чехлы, обложки для гаджетов (телефонов, планшетов etc)",
    "портативная акустика",
    "мобильные телефоны",
    "VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)",
    "планшетные компьютеры и мини-планшеты",
    "наушники, гарнитуры, наушники c микрофоном",
    "радиоприемники, радиобудильники, радиочасы",
    "магнитолы",
    "GPS-навигаторы"
    ]

categories_id = [
    3902,
    510402,
    4302,
    2815,
    3901,
    # 3994,
    3904,
    2801,
    3908,
    510401,
    2102,
    3903,
    3907,
    280801
    ]

dst_fld = "/home/sondors/Documents/price/ColBERT/tmp_tutorial"
for cat_id in categories_id:
    category_models = df_models[df_models.category_id == cat_id].reset_index(drop=True)
    prepare_tsv(category_models, dst_fld, cat_id)

### Индексируем модели и сохраняем индекс на диск

In [4]:
def save_index(ckpt_pth, doc_maxlen, nbits, nranks, experiment, collection, index_name):
    with Run().context(RunConfig(nranks=nranks, experiment=experiment)):
        config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits)#, root="/path/to/experiments")
        indexer = Indexer(checkpoint=ckpt_pth, config=config)
        indexer.index(name=index_name, collection=collection, overwrite=True)
    return indexer

ckpt_pth = "/home/sondors/Documents/ColBERT_weights/bert-base-multilingual-cased_dim_768_bsize_230_lr04_use_ib_negatives/none/2024-01/27/16.55.29/checkpoints/colbert-2998-finish"
experiment = "tutorial"

doc_maxlen = 300
nbits = 2   # encode each dimension with 2 bits
nranks = 1  # nranks specifies the number of GPUs to use.

for cat_id in categories_id:

    models_colbert = Collection(path=os.path.join(dst_fld,f"{cat_id}_models.tsv"))
    index_name = f'models_{cat_id}_{nbits}bits'
    indexer = save_index(ckpt_pth, doc_maxlen, nbits, nranks, experiment, models_colbert, index_name)

[Feb 06, 17:49:23] #> Loading collection...
0M 


[Feb 06, 17:49:23] #> Note: Output directory /home/sondors/Documents/price/ColBERT/experiments/tutorial/indexes/models_3902_2bits already exists


[Feb 06, 17:49:23] #> Will delete 10 files already at /home/sondors/Documents/price/ColBERT/experiments/tutorial/indexes/models_3902_2bits in 20 seconds...
#> Starting...
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "nbits": 2,
    "kmeans_niters": 4,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 0.0001,
    "maxsteps": 2998,
    "save_every": null,
    "warmup": 0,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": fals

/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
  0%|          | 0/8 [00:00<?, ?it/s]/home/sondors/anaconda3/envs/colbert_cpu/lib/python3.8/site-packages/torch/amp/autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[Feb 06, 17:49:46] [0] 		 # of sampled PIDs = 488 	 sampled_pids[:3] = [213, 375, 5]
[Feb 06, 17:49:46] [0] 		 #> Encoding 488 passages..


100%|██████████| 8/8 [00:03<00:00,  2.20it/s]
WARNING clustering 4691 points to 1024 centroids: please provide at least 39936 training points
0it [00:00, ?it/s]
  0%|          | 0/8 [00:00<?, ?it/s]

[Feb 06, 17:49:50] [0] 		 avg_doclen_est = 10.116803169250488 	 len(local_sample) = 488
[Feb 06, 17:49:50] [0] 		 Creaing 1,024 partitions.
[Feb 06, 17:49:50] [0] 		 *Estimated* 4,936 embeddings.
[Feb 06, 17:49:50] [0] 		 #> Saving the indexing plan to /home/sondors/Documents/price/ColBERT/experiments/tutorial/indexes/models_3902_2bits/plan.json ..
Clustering 4691 points in 768D to 1024 clusters, redo 1 times, 4 iterations
  Preprocessing in 0.00 s
  Iteration 3 (0.07 s, search 0.06 s): objective=392.74 imbalance=1.505 nsplit=0        
[0.009, 0.008, 0.01, 0.007, 0.011, 0.008, 0.01, 0.01, 0.009, 0.009, 0.008, 0.009, 0.008, 0.007, 0.008, 0.008, 0.01, 0.008, 0.01, 0.008, 0.009, 0.01, 0.009, 0.009, 0.008, 0.011, 0.008, 0.008, 0.007, 0.009, 0.01, 0.01, 0.008, 0.009, 0.009, 0.009, 0.008, 0.008, 0.009, 0.007, 0.011, 0.009, 0.009, 0.01, 0.01, 0.009, 0.008, 0.008, 0.007, 0.007, 0.009, 0.007, 0.009, 0.008, 0.008, 0.008, 0.008, 0.01, 0.009, 0.007, 0.009, 0.007, 0.009, 0.01, 0.008, 0.008, 0.008, 

...

0it [00:00, ?it/s]
100%|██████████| 14/14 [00:05<00:00,  2.37it/s]


[Feb 06, 17:52:57] [0] 		 #> Saving chunk 0: 	 847 passages and 8,812 embeddings. From #0 onward.
[Feb 06, 17:52:57] [0] 		 #> Checking all files were saved...
[Feb 06, 17:52:57] [0] 		 Found all files!
[Feb 06, 17:52:57] [0] 		 #> Building IVF...
[Feb 06, 17:52:57] [0] 		 #> Loading codes...
[Feb 06, 17:52:57] [0] 		 Sorting codes...
[Feb 06, 17:52:57] [0] 		 Getting unique codes...
[Feb 06, 17:52:57] #> Optimizing IVF to store map from centroids to list of pids..
[Feb 06, 17:52:57] #> Building the emb2pid mapping..
[Feb 06, 17:52:57] len(emb2pid) = 8812
[Feb 06, 17:52:57] #> Saved optimized IVF to /home/sondors/Documents/price/ColBERT/experiments/tutorial/indexes/models_3901_2bits/ivf.pid.pt
[Feb 06, 17:52:57] [0] 		 #> Saving the indexing metadata to /home/sondors/Documents/price/ColBERT/experiments/tutorial/indexes/models_3901_2bits/metadata.json ..


1it [00:06,  6.04s/it]
100%|██████████| 1024/1024 [00:00<00:00, 158147.41it/s]


#> Joined...
[Feb 06, 17:52:58] #> Loading collection...
0M 


[Feb 06, 17:52:58] #> Note: Output directory /home/sondors/Documents/price/ColBERT/experiments/tutorial/indexes/models_3904_2bits already exists


[Feb 06, 17:52:58] #> Will delete 10 files already at /home/sondors/Documents/price/ColBERT/experiments/tutorial/indexes/models_3904_2bits in 20 seconds...


KeyboardInterrupt: 

### Поиск матча по индексу

In [34]:
def top_n_similar(offers, experiment, index_name, model_ids, n):
    with Run().context(RunConfig(experiment=experiment)):
        # config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits)#, root="/path/to/experiments")
        searcher = Searcher(index=index_name, collection=model_ids)#, config=config)
        offers = Queries(data=offers)
        rankings = searcher.search_all(offers, k=n)
        top_n = rankings_to_dict(rankings, searcher)
    return top_n

def rankings_to_dict(rankings, searcher):
    result = []
    for key, value in rankings.todict().items():
        model_ids = [int(searcher.collection[item[0]]) for item in value]
        similarity = [item[2] for item in value]
        result.append({'model_ids': model_ids, 'similarity': similarity})
    return result

offers = {
    0: 'Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)',
    1: 'Планшет Samsung Galaxy Tab S8 128GB 5G Silver (SM-X706B)',
    2: 'Планшет Samsung Galaxy Tab S8+ 128GB Wi-Fi Pink Gold (SM-X800)'
    }

experiment = "tutorial"
doc_maxlen = 300
nbits = 2   # encode each dimension with 2 bits
nranks = 1  # nranks specifies the number of GPUs to use.
cat_id = 2801 # мобильные телефоны
cat_id = 510401 # планшеты
index_name = f'models_{cat_id}_{nbits}bits'
models_id_colbert = Collection(path=os.path.join(dst_fld,f"{cat_id}_models_id.tsv"))

top_n = top_n_similar(offers, experiment, index_name, models_id_colbert, 5)

[Feb 06, 18:54:21] #> Loading collection...
0M 
[Feb 06, 18:54:22] #> Loading codec...
[Feb 06, 18:54:22] #> Loading IVF...
[Feb 06, 18:54:22] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 2264.74it/s]

[Feb 06, 18:54:22] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 320.74it/s]



#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global), 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([   101,    100,  37077,    524,  19079, 105694,  37077,  29723,  14248,
         10457,    156,  11396,    117,    129,    512,  18683,    120,  16196,
           512,  18683,    117,  52742,    118,  36448,    116,  29494,  18062,
           117,  10956,  67459,  19954,    102])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])



3it [00:00, 355.64it/s]


### Получаем выдачу топ N моделей для каждого оффера

In [32]:
for i in range(len(top_n)):
    print(offers[i])
    for j in range(len(top_n[i]['model_ids'])):
        id = top_n[i]['model_ids'][j]
        sim = top_n[i]['similarity'][j]
        model = list(df_models[df_models.model_id == id]['full_name'])[0]
        print(f"\t{id}: {model} --> {round(float(sim), 2)}")
    print("_"*60)

Samsung Планшет Samsung Galaxy Tab S8, 8 ГБ/128 ГБ, Wi-Fi + Cellular, со стилусом, графит (Global)
	5144478: Samsung Galaxy Tab S8 --> 28.97
	5144477: Samsung Galaxy Tab S8+ --> 24.97
	937550: Samsung Galaxy Tab S2 9.7 SM-T813 --> 24.0
	5144479: Samsung Galaxy Tab S8 Ultra --> 23.98
	816746: Samsung Galaxy Tab S2 9.7 SM-T810 --> 23.64
____________________________________________________________
Планшет Samsung Galaxy Tab S8 128GB 5G Silver (SM-X706B)
	5144478: Samsung Galaxy Tab S8 --> 28.15
	5144477: Samsung Galaxy Tab S8+ --> 23.94
	937550: Samsung Galaxy Tab S2 9.7 SM-T813 --> 23.21
	623216: Samsung Galaxy Tab S 8.4 SM-T705 --> 22.82
	816746: Samsung Galaxy Tab S2 9.7 SM-T810 --> 22.65
____________________________________________________________
Планшет Samsung Galaxy Tab S8+ 128GB Wi-Fi Pink Gold (SM-X800)
	5144477: Samsung Galaxy Tab S8+ --> 26.03
	5144478: Samsung Galaxy Tab S8 --> 20.32
	4509798: Samsung Galaxy Tab S7+ 12.4 128Gb --> 18.63
	937550: Samsung Galaxy Tab S2 9.7 SM-T

In [33]:
print(top_n)

[{'model_ids': [5144478, 5144477, 937550, 5144479, 816746], 'similarity': [28.973285675048828, 24.965259552001953, 23.999181747436523, 23.97533416748047, 23.638072967529297]}, {'model_ids': [5144478, 5144477, 937550, 623216, 816746], 'similarity': [28.152374267578125, 23.93535041809082, 23.20758819580078, 22.81966781616211, 22.648759841918945]}, {'model_ids': [5144477, 5144478, 4509798, 937550, 5144479], 'similarity': [26.02865982055664, 20.31949234008789, 18.63184928894043, 18.34659194946289, 18.30866813659668]}]
